# Explore here

In [6]:
# Your code here
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,False
2995,https://www.youtube.com/watch,True
2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2997,https://www.technologyreview.com/2019/12/20/13...,False


In [7]:
df.shape

(2999, 2)

In [8]:
df.duplicated().sum()

np.int64(630)

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.duplicated().sum()

np.int64(0)

In [11]:
df.reset_index()

,index,url,is_spam
0,0,https://briefingday.us8.list-manage.com/unsubs...,True
1,1,https://www.hvper.com/,True
2,2,https://briefingday.com/m/v4n3i4f3,True
3,3,https://briefingday.com/n/20200618/m#commentform,False
4,4,https://briefingday.com/fan,True
...,...,...,...
2364,2993,https://www.theverge.com/2020/6/29/21306889/di...,False
2365,2994,https://www.smartcitiesworld.net/news/news/dee...,False
2366,2996,https://techcrunch.com/2019/07/04/an-optimisti...,False
2367,2997,https://www.technologyreview.com/2019/12/20/13...,False


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2369 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2369 non-null   object
 1   is_spam  2369 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 39.3+ KB


In [13]:
# Revisar el balance de las variables, cuantas son true y false
# Esta bastente desbalanceada

print(len(df[df['is_spam']==True]))
print(len(df[df['is_spam']==False]))

244
2125


In [14]:
import regex as re  

def preprocess_text(text):
    # Remove any character that is not a letter (a-z) or white space ( )
    text = re.sub(r'[^a-z ]', " ", text)
    
    # Remove white spaces
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    # Multiple white spaces into one
    text = re.sub(r'\s+', " ", text.lower())

    return text

print(preprocess_text(df.url[0]))





https briefingday us list manage com unsubscribe


Acá se eliminaron /,: y otros para que quede todo separado ' '

In [15]:
#df.url es lo mismo que df['url']

df.url = df.url.apply(preprocess_text)

df

,url,is_spam
0,https briefingday us list manage com unsubscribe,True
1,https www hvper com,True
2,https briefingday com v i,True
3,https briefingday com m commentform,False
4,https briefingday com fan,True
...,...,...
2993,https www theverge com disney deepfake face sw...,False
2994,https www smartcitiesworld net news news deepf...,False
2996,https techcrunch com an optimistic view of dee...,False
2997,https www technologyreview com this startup cl...,False
